In [1]:
import sys
from typing import List, Dict, Any, Optional

import tiktoken
import pandas as pd
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings
from llama_index.core import SummaryIndex, VectorStoreIndex
from llama_index.readers.web import SimpleWebPageReader
from llama_index.core.tools import BaseTool, FunctionTool
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.query_engine import SubQuestionQueryEngine
from llama_index.core.agent import ReActAgent
from llama_index.core.response_synthesizers import TreeSummarize
from llama_index.core.query_pipeline import QueryPipeline, FnComponent
from llama_index.core import PromptTemplate
from llama_index.core import get_response_synthesizer
from llama_index.core.response_synthesizers import ResponseMode
from llama_index.core.output_parsers import PydanticOutputParser
from llama_index.core.callbacks import CallbackManager, TokenCountingHandler
from pydantic import BaseModel, Field, conlist, field_validator
from enum import Enum
from IPython.display import Markdown, display
from duckduckgo_search import DDGS

sys.path.append('../')

from src.generators import (
    prepare_regulation_syllabus_pipeline,
    prepare_regulation_query_engine,
    prepare_section_judge_pipeline,
)
from src.prompts import (
    ASK_SYLLABUS_TEMPLATE,
    FORMAT_SYLLABUS_TEMPLATE,
    JUDGE_SECTION_TEMPLATE,
    FORMAT_JUDGE_TEMPLATE,
)
from src.formats import (
    Syllabus,
    SectionNames,
    Judges,
    Judge,
)
from src.law_model import prepare_law_llm

d:\Program\miniconda3\envs\DL\lib\site-packages\pydantic\_internal\_fields.py:160: UserWarning: Field "model_id" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
d:\Program\miniconda3\envs\DL\lib\site-packages\transformers\utils\generic.py:485: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [2]:
import nest_asyncio

nest_asyncio.apply()

In [3]:
llm = OpenAI(model="gpt-4o")
Settings.llm = llm

In [4]:
tokenizer_fn = tiktoken.encoding_for_model("gpt-4o").encode
token_counter = TokenCountingHandler(tokenizer=tokenizer_fn, verbose=True)
Settings.callback_manager = CallbackManager([token_counter])

## Syllabus Generator

In [5]:
links_df = pd.read_csv('../data/regulations/links.csv', encoding='utf-8')
regulation_query_engine = prepare_regulation_query_engine(links_df)
syllabus_generator = prepare_regulation_syllabus_pipeline(
    regulation_query_engine, verbose=True)

In [6]:
output = syllabus_generator.run(regulations="GDPR, PIPEDA")
output

> Running module 2e71829b-6511-43d3-8fa6-1338ce3eb6eb with input: 
regulations: GDPR, PIPEDA

> Running module cb46a61a-cb1c-410b-b29b-bda48156b929 with input: 
input: I want to write a privacy policy, which should be GDPR, PIPEDA compliant.
What sections should it include? And for each section, what key points should be covered?
List them as bullet points.

LLM Prompt Token Usage: 470
LLM Completion Token Usage: 137
Generated 4 sub questions.
[General Data Protection Regulation (GDPR)] Q: What sections should a privacy policy include to be GDPR compliant?
[General Data Protection Regulation (GDPR)] Q: What key points should be covered in each section of a GDPR compliant privacy policy?
[Personal Information Protection and Electronic Documents Act (PIPEDA)] Q: What sections should a privacy policy include to be PIPEDA compliant?
[Personal Information Protection and Electronic Documents Act (PIPEDA)] Q: What key points should be covered in each section of a PIPEDA compliant privacy poli

{'Introduction': ['Purpose of the privacy policy.',
  "Overview of the organization's commitment to data protection."],
 'Data Controller Information': ['Identify the individual(s) responsible for ensuring compliance with GDPR and PIPEDA principles within the organization.',
  'Contact details of the data protection officer, if applicable.'],
 'Data Collection and Usage': ['Clearly state the purposes for which personal information is being collected, used, and disclosed.',
  'Categories of personal data collected (e.g., contact details, financial information).',
  'Sources of personal data (e.g., directly from users, third parties).',
  'Legal basis for each processing activity (e.g., consent, contractual necessity, legal obligation).',
  'Ensure that the collection of personal information is limited to what is necessary for the identified purposes and is collected by fair and lawful means.',
  'Describe how personal information will be used and disclosed only for the purposes for whic

## Judge Generator

In [5]:
generated_section = """\
The data we collect depends on the context of your interactions with Microsoft and the choices you make (including your privacy settings), the products and features you use, your location, and applicable law.

The data we collect can include the following:

Name and contact data. Your first and last name, email address, postal address, phone number, and other similar contact data.

Credentials. Passwords, password hints, and similar security information used for authentication and account access.

Demographic data. Data about you such as your age, gender, country, and preferred language.

Payment data. Data to process payments, such as your payment instrument number (such as a credit card number) and the security code associated with your payment instrument.

Subscription and licensing data. Information about your subscriptions, licenses, and other entitlements.

Microsoft uses the data we collect to provide you rich, interactive experiences. In particular, we use data to:

Provide our products, which includes updating, securing, and troubleshooting, as well as providing support. It also includes sharing data, when it is required to provide the service or carry out the transactions you request.
Improve and develop our products.
Personalize our products and make recommendations.
Advertise and market to you, which includes sending promotional communications, targeting advertising, and presenting you relevant offers.
We also use the data to operate our business, which includes analyzing our performance, meeting our legal obligations, developing our workforce, and doing research."""

In [6]:
law_llm = prepare_law_llm("Equall/Saul-7B-Instruct-v1")

d:\Program\miniconda3\envs\DL\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
d:\Program\miniconda3\envs\DL\lib\site-packages\transformers\utils\generic.py:342: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
d:\Program\miniconda3\envs\DL\lib\site-packages\transformers\utils\generic.py:342: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

d:\Program\miniconda3\envs\DL\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [7]:
judge_pipeline = prepare_section_judge_pipeline(law_llm, verbose=True)

In [8]:
output = judge_pipeline.run(
    section_name="Data Collection and Usage",
    section_text=generated_section,
    regulations="GDPR, CCPA",
)

> Running module judge_section_template with input: 
section_name: Data Collection and Usage
section_text: The data we collect depends on the context of your interactions with Microsoft and the choices you make (including your privacy settings), the products and features you use, your location, and applica...
regulations: GDPR, CCPA

> Running module law_llm with input: 
prompt: This Data Collection and Usage is a section of a privacy policy:

The data we collect depends on the context of your interactions with Microsoft and the choices you make (including your privacy settin...



Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


LLM Prompt Token Usage: 357
LLM Completion Token Usage: 233
> Running module format_judge_template with input: 
query_str: This section appears to be compliant with the GDPR and CCPA regulations. It provides clear information about the data collected by Microsoft, the purposes for which the data is used, and the legal bas...

> Running module llm with input: 
messages: Given the comments from a legal expert:

This section appears to be compliant with the GDPR and CCPA regulations. It provides clear information about the data collected by Microsoft, the purposes for ...

LLM Prompt Token Usage: 521
LLM Completion Token Usage: 175
> Running module judge_parser with input: 
input: assistant: ```json
{
  "judges": [
    {
      "name": "Data Collection and Usage",
      "suggestions": "Clarify the legal basis for processing each type of data collected, such as consent, contract,...

> Running module determine_judge with input: 
prompt: This Data Collection and Usage is a section of a priva

In [9]:
output

{'pass': False,
 'suggestions': 'Clarify the legal basis for processing each type of data collected, such as consent, contract, or legitimate interest.'}

## Save links

In [11]:
law_links = {
    'regulations': [
        'General Data Protection Regulation (GDPR)',
        'California Consumer Privacy Act (CCPA)',
        'Personal Information Protection and Electronic Documents Act (PIPEDA)'
    ],
    'links': [
        'https://eur-lex.europa.eu/legal-content/EN/TXT/HTML/?uri=CELEX:32016L0680',
        'https://leginfo.legislature.ca.gov/faces/codes_displayText.xhtml?division=3.&part=4.&lawCode=CIV&title=1.81.5',
        'https://laws-lois.justice.gc.ca/ENG/ACTS/P-8.6/FullText.html'
    ]
}

In [12]:
pd.DataFrame(law_links).to_csv('../data/regulations/links.csv',
                               index=False,
                               encoding='utf-8')

## To QueryEngines

In [4]:
links_df = pd.read_csv('../data/regulations/links.csv', encoding='utf-8')
links_df

,regulations,links
0,General Data Protection Regulation (GDPR),https://eur-lex.europa.eu/legal-content/EN/TXT...
1,California Consumer Privacy Act (CCPA),https://leginfo.legislature.ca.gov/faces/codes...
2,Personal Information Protection and Electronic...,https://laws-lois.justice.gc.ca/ENG/ACTS/P-8.6...


In [5]:
regulation_text = SimpleWebPageReader(html_to_text=True).load_data(
    links_df['links'].to_list())

In [6]:
# regulation_query_tools = [
#     QueryEngineTool(
#         query_engine=VectorStoreIndex.from_documents(
#             [regulation_text[i]]).as_query_engine(llm=llm),
#         metadata=ToolMetadata(
#             name=links_df.loc[i, 'regulations'],
#             description=("refer the text when you want to make something be "
#                          f"{links_df.loc[i, 'regulations']} compliant"),
#         ),
#     ) for i in range(len(regulation_text))
# ]

In [6]:
regulation_query_tools = [
    QueryEngineTool(
        query_engine=SummaryIndex.from_documents([
            regulation_text[i]
        ]).as_query_engine(response_mode="tree_summarize", llm=llm),
        metadata=ToolMetadata(
            name=links_df.loc[i, 'regulations'],
            description=("refer the text when you want to make something be "
                         f"{links_df.loc[i, 'regulations']} compliant"),
        ),
    ) for i in range(len(regulation_text))
]

In [7]:
regulation_query_engine = SubQuestionQueryEngine.from_defaults(
    query_engine_tools=regulation_query_tools, llm=llm, verbose=True)

## Syllabus Generator (RAW)

In [6]:
ask_syllabus_template = PromptTemplate(ASK_SYLLABUS_TEMPLATE)

In [6]:
class SectionNames(str, Enum):
    SECTION1 = "Introduction"
    SECTION2 = "Data Controller Information"
    SECTION3 = "Data Collection and Usage"
    SECTION4 = "Data Subject Rights"
    SECTION5 = "Data Sharing and Transfers"
    SECTION6 = "Data Retention"
    SECTION7 = "Disclosure of Personal Information"
    SECTION8 = "Security Measures"
    SECTION9 = "Automated Decision-Making and Profiling"
    SECTION10 = "Cookies and Tracking Technologies"
    SECTION11 = "Changes to the Privacy Policy"
    SECTION12 = "Contact Information"


class Section(BaseModel):
    name: SectionNames = Field(..., title="Section Name")
    key_points: List[str] = Field(..., title="List of Key Points")


SectionList = conlist(Section, min_length=12, max_length=12)


class Syllabus(BaseModel):
    sections: SectionList = Field(..., title="List of Sections")

In [7]:
output_parser = PydanticOutputParser(SyllabusFormat)
split_section_template = PromptTemplate(
    output_parser.format(FORMAT_SYLLABUS_TEMPLATE))


In [14]:
print(split_section_template.template)

Given the privacy policy sections and key points:

{query_str}

DO NOT modify the key points!!!
Please rearrange the sections and key points and output with the following JSON format:


Here's a JSON schema to follow:
{{"$defs": {{"Section": {{"properties": {{"name": {{"allOf": [{{"$ref": "#/$defs/SectionNames"}}], "title": "Section Name"}}, "key_points": {{"items": {{"type": "string"}}, "title": "List of Key Points", "type": "array"}}}}, "required": ["name", "key_points"], "title": "Section", "type": "object"}}, "SectionNames": {{"enum": ["Introduction", "Data Controller Information", "Data Collection and Usage", "Data Subject Rights", "Data Sharing and Transfers", "Data Retention", "Disclosure of Personal Information", "Security Measures", "Automated Decision-Making and Profiling", "Cookies and Tracking Technologies", "Changes to the Privacy Policy", "Contact Information"], "title": "SectionNames", "type": "string"}}}}, "properties": {{"sections": {{"items": {{"$ref": "#/$defs/Sectio

In [8]:
syllabus_generator.module_dict[list(syllabus_generator.module_dict.keys(
))[2]].prompt.template == split_section_template.template

True

In [9]:
p = QueryPipeline(chain=[
    ask_syllabus_template,
    regulation_query_engine,
    split_section_template,
    llm,
    output_parser,
],
                  verbose=True)

In [13]:
output = p.run(regulations="GDPR, PIPEDA")
# display(Markdown(f"<b>{output.response}</b>"))

> Running module b637576c-0721-486a-9359-7f481394a8d9 with input: 
regulations: GDPR, PIPEDA

> Running module c6be2272-cd88-43cf-be35-48299779b46a with input: 
input: I want to write a privacy policy, which should be GDPR, PIPEDA compliant. 
    What sections should it include? And for each section, what key points should be covered?
    List them as bullet points....

Generated 4 sub questions.
[General Data Protection Regulation (GDPR)] Q: What sections should a GDPR compliant privacy policy include?
[General Data Protection Regulation (GDPR)] Q: For each section of a GDPR compliant privacy policy, what key points should be covered?
[Personal Information Protection and Electronic Documents Act (PIPEDA)] Q: What sections should a PIPEDA compliant privacy policy include?
[Personal Information Protection and Electronic Documents Act (PIPEDA)] Q: For each section of a PIPEDA compliant privacy policy, what key points should be covered?
[General Data Protection Regulation (GDPR)] A: A GDP

In [60]:
output.sections

[Section(name=<SectionNames.SECTION1: 'Introduction'>, key_points=['Purpose of the privacy policy.', "Overview of the organization's commitment to data protection."]),
 Section(name=<SectionNames.SECTION2: 'Data Controller Information'>, key_points=['Identity and contact details of the data controller.', 'Contact details of the Data Protection Officer (if applicable).']),
 Section(name=<SectionNames.SECTION3: 'Data Collection and Usage'>, key_points=['Categories of personal data collected (e.g., contact details, financial information).', 'Methods of data collection (e.g., directly from users, through cookies).', 'Specific purposes for which personal data is processed.', 'Legal basis for each processing activity (e.g., consent, performance of a contract, legal obligation).', "Obtain the individual's knowledge and consent for the collection, use, or disclosure of personal information, except where inappropriate.", 'Ensure consent is meaningful by explaining the purposes in a manner that 

In [61]:
for i, section in enumerate(output.sections):
    display(
        Markdown(f"<b>**{i+1}. {section.name.value}**\n\n&ensp;- " +
                 '\n\n&ensp;- '.join(section.key_points) + "</b>"))

<b>**1. Introduction**

&ensp;- Purpose of the privacy policy.

&ensp;- Overview of the organization's commitment to data protection.</b>

<b>**2. Data Controller Information**

&ensp;- Identity and contact details of the data controller.

&ensp;- Contact details of the Data Protection Officer (if applicable).</b>

<b>**3. Data Collection and Usage**

&ensp;- Categories of personal data collected (e.g., contact details, financial information).

&ensp;- Methods of data collection (e.g., directly from users, through cookies).

&ensp;- Specific purposes for which personal data is processed.

&ensp;- Legal basis for each processing activity (e.g., consent, performance of a contract, legal obligation).

&ensp;- Obtain the individual's knowledge and consent for the collection, use, or disclosure of personal information, except where inappropriate.

&ensp;- Ensure consent is meaningful by explaining the purposes in a manner that individuals can understand.

&ensp;- Limit the collection of personal information to what is necessary for the identified purposes.

&ensp;- Collect information by fair and lawful means.</b>

<b>**4. Data Subject Rights**

&ensp;- Right to access personal data.

&ensp;- Right to rectification, erasure, and restriction of processing.

&ensp;- Right to data portability.

&ensp;- Right to object to processing.

&ensp;- Right to withdraw consent at any time.</b>

<b>**5. Data Sharing and Transfers**

&ensp;- Categories of recipients with whom personal data is shared.

&ensp;- Information on international data transfers and safeguards in place (e.g., adequacy decisions, standard contractual clauses).</b>

...

<b>**8. Security Measures**

&ensp;- Description of technical and organizational measures to protect personal data.

&ensp;- Procedures in place to handle data breaches.</b>

<b>**9. Automated Decision-Making and Profiling**

&ensp;- Information on any automated decision-making, including profiling.

&ensp;- Significance and consequences of such processing for data subjects.</b>

<b>**10. Cookies and Tracking Technologies**

&ensp;- Types of cookies and tracking technologies used.

&ensp;- Purpose of using cookies.

&ensp;- How users can manage cookie preferences.</b>

<b>**11. Changes to the Privacy Policy**

&ensp;- How and when changes to the privacy policy will be communicated to users.

&ensp;- Effective date of the current privacy policy.</b>

<b>**12. Contact Information**

&ensp;- Information on how to lodge a complaint with the organization.

&ensp;- Contact details of the relevant supervisory authority for data protection issues.

&ensp;- Provide mechanisms for individuals to challenge the organization's compliance with the above principles.

&ensp;- Investigate and respond to complaints regarding the handling of personal information.</b>

In [27]:
output.sections[0].key_points

['Purpose of the privacy policy.',
 "Overview of the organization's commitment to data protection.",
 'Scope of the privacy policy.']

In [11]:
output = regulation_query_engine.query(
    """I want to write a privacy policy, which should be GDPR and CCPA compliant.
    What sections should it include? And for each section, what key points should be covered?
    List them as bullet points.
    """)
display(Markdown(f"<b>{output.response}</b>"))

Generated 4 sub questions.
[General Data Protection Regulation (GDPR)] Q: What sections should a GDPR compliant privacy policy include?
[General Data Protection Regulation (GDPR)] Q: For each section of a GDPR compliant privacy policy, what key points should be covered?
[California Consumer Privacy Act (CCPA)] Q: What sections should a CCPA compliant privacy policy include?
[California Consumer Privacy Act (CCPA)] Q: For each section of a CCPA compliant privacy policy, what key points should be covered?
[General Data Protection Regulation (GDPR)] A: A GDPR compliant privacy policy should include the following sections:

1. **Introduction**: An overview of the policy and its purpose.
2. **Data Controller Information**: Details about the entity responsible for data processing.
3. **Data Protection Officer**: Contact information if a Data Protection Officer is appointed.
4. **Types of Data Collected**: Description of the personal data being collected.
5. **Purpose of Data Processing**: Ex

<b>To write a privacy policy that is both GDPR and CCPA compliant, you should include the following sections and cover the key points for each:

1. **Introduction**:
   - Purpose of the privacy policy.
   - Overview of the organization's commitment to data protection.
   - Overview of GDPR and CCPA compliance.

2. **Data Controller Information**:
   - Name and contact details of the data controller.
   - Contact details of the Data Protection Officer (DPO), if applicable.

3. **Categories of Personal Information Collected**:
   - Types of personal information collected (e.g., identifiers, contact details, financial information, biometric information).
   - Methods of data collection (e.g., directly from users, through website forms).

4. **Sources of Personal Information**:
   - Categories of sources from which personal information is collected.

5. **Purpose of Data Processing**:
   - Specific purposes for which personal data is processed (e.g., service provision, marketing, legal compliance).
   - Legal basis for processing (e.g., consent, contractual necessity, legal obligation).
   - Whether the information is sold or shared.

6. **Legal Basis for Processing**:
   - The legal grounds for processing personal data under GDPR.

7. **Data Subject and Consumer Rights**:
   - Rights of data subjects under GDPR (e.g., access, rectification, erasure, restriction of processing, data portability, objection).
   - Rights of consumers under CCPA (e.g., right to know, access, delete, correct inaccurate personal information, opt-out of sale or sharing, limit use of sensitive personal information, no retaliation).
   - How data subjects and consumers can exercise their rights.

8. **Data Retention**:
   - Criteria for determining data retention periods.
   - Specific retention periods for different types of data, if applicable.

9. **Data Sharing and Recipients**:
   - Categories of recipients with whom data may be shared (e.g., service providers, business partners).
   - Information on international data transfers and safeguards in place.

10. **Security Measures**:
    - Description of technical and organizational measures in place to protect personal data.
    - Procedures for handling data breaches.

11. **Automated Decision-Making**:
    - Information on any automated decision-making processes, including profiling.

12. **Cookies and Tracking Technologies**:
    - Types of cookies and tracking technologies used.
    - Purpose of using cookies and how users can manage their cookie preferences.

13. **Financial Incentives**:
    - Description of any financial incentives offered in exchange for personal information.
    - Terms of the financial incentive program and how consumers can opt-in or opt-out.

14. **Non-Discrimination**:
    - Statement that the business will not discriminate against consumers for exercising their CCPA rights.

15. **Complaints**:
    - How to lodge a complaint with a supervisory authority.

16. **Changes to the Privacy Policy**:
    - How and when changes to the privacy policy will be communicated to users.
    - Effective date of the current privacy policy.

17. **Contact Information**:
    - How users can contact the organization for privacy-related inquiries or complaints.
    - Methods for consumers to submit requests regarding their personal information, including at least one designated method such as a toll-free telephone number or an email address.

Each section should be written in clear and plain language to ensure that it is easily understandable by the data subjects and consumers.</b>

In [9]:
output = regulation_query_engine.query(
    """I want to write a privacy policy, which should be GDPR and CCPA compliant.
    
    It includes the following sections:
    1. Introduction
    2. Identity and Contact Details of the Controller
    3. Data Collection and Use
    4. Data Subject Rights
    5. Data Sharing and Transfers
    6. Data Retention
    7. Security Measures
    8. Automated Decision-Making and Profiling
    9. Cookies and Tracking Technologies
    10. Changes to the Privacy Policy
    11. Contact Information

    Now given the sections, I want to know what should be included in each section to be compliant with GDPR and CCPA.
    List them as bullet points.
    """)
display(Markdown(f"<b>{output.response}</b>"))

Generated 22 sub questions.
[General Data Protection Regulation (GDPR)] Q: What should be included in the Introduction section to be GDPR compliant?
[General Data Protection Regulation (GDPR)] Q: What should be included in the Identity and Contact Details of the Controller section to be GDPR compliant?
[General Data Protection Regulation (GDPR)] Q: What should be included in the Data Collection and Use section to be GDPR compliant?
[General Data Protection Regulation (GDPR)] Q: What should be included in the Data Subject Rights section to be GDPR compliant?
[General Data Protection Regulation (GDPR)] Q: What should be included in the Data Sharing and Transfers section to be GDPR compliant?
[General Data Protection Regulation (GDPR)] Q: What should be included in the Data Retention section to be GDPR compliant?
[General Data Protection Regulation (GDPR)] Q: What should be included in the Security Measures section to be GDPR compliant?
[General Data Protection Regulation (GDPR)] Q: What 

<b>To ensure your privacy policy is compliant with both GDPR and CCPA, each section should include the following elements:

### 1. Introduction
- **GDPR**:
  - Identity of the data controller.
  - Existence of the data processing operation.
  - Purposes of the data processing.
  - Right of the data subject to lodge a complaint.
  - Right of the data subject to request access to, rectification or erasure of personal data, or restriction of processing.
- **CCPA**:
  - Categories of personal information collected.
  - Purposes for collection.
  - Disclosure of sale or sharing of personal information.
  - Categories and purposes of sensitive personal information collected.
  - Retention period for each category of personal information.
  - Opt-out information.
  - Accessibility for consumers with disabilities and in the primary language used to interact with the consumer.

### 2. Identity and Contact Details of the Controller
- **GDPR**:
  - Identity of the controller.
  - Existence of the processing operation.
  - Purposes of the processing.
  - Right to lodge a complaint.
  - Right to request access to and rectification or erasure of personal data or restriction of processing.
  - Legal basis for the processing.
  - Data storage duration.
- **CCPA**:
  - Name or names of the owner, director, officer, or management employee.
  - Contact information, including business title.

### 3. Data Collection and Use
- **GDPR**:
  - Identity of the controller.
  - Existence of the processing operation.
  - Purposes of the processing.
  - Legal basis for processing.
  - Categories of personal data.
  - Data subject rights.
  - Safeguards for sensitive data.
  - Time limits for data storage.
  - Quality and accuracy of data.
  - Automated decision-making information.
  - Right to lodge a complaint.
  - Additional information for identity verification.
- **CCPA**:
  - Categories of personal information collected.
  - Categories of sensitive personal information collected.
  - Retention period for each category of personal information.
  - Purpose of collection and use.
  - Third-party agreements.
  - Security measures.
  - Consumer rights to request deletion.
  - Opt-out options.

### 4. Data Subject Rights
- **GDPR**:
  - Identity of the controller.
  - Existence of the processing operation.
  - Purposes of the processing.
  - Right to lodge a complaint.
  - Right to request access to and rectification or erasure of personal data or restriction of processing.
  - Legal basis for the processing.
  - Data storage duration.
  - How to exercise these rights.
- **CCPA**:
  - Right to request correction of inaccurate personal information.
  - Right to know what personal information is being collected.
  - Right to opt-out of the sale or sharing of personal information.
  - Right to limit the use or disclosure of sensitive personal information.
  - How to exercise these rights.
  - Disclosure of financial incentives.

### 5. Data Sharing and Transfers
- **GDPR**:
  - Conditions for transfers to third countries or international organizations.
  - Protections for sensitive personal data.
  - Automated decision-making safeguards.
  - Information accessibility.
  - Mechanisms for data subject rights.
  - Identity confirmation procedures.
  - Mandatory information.
  - Adequacy decisions.
  - Periodic review and monitoring.
  - Consultation and remediation procedures.
  - Appropriate safeguards for non-adequacy decision transfers.
- **CCPA**:
  - Identification of the consumer and association of information.
  - Categories of personal information sold or shared.
  - Categories of personal information disclosed for a business purpose.
  - Description of consumer rights and methods for submitting requests.
  - Lists of categories of personal information collected, sold, shared, and disclosed.
  - Opt-out information.
  - Terms of financial incentives.

### 6. Data Retention
- **GDPR**:
  - Purpose limitation.
  - Time limits for data retention.
  - Review procedures.
  - Legal basis for processing and storage duration.
  - Data subject rights.
  - Compliance responsibility.
- **CCPA**:
  - Retention period for each category of personal information.
  - Criteria used to determine retention period.
  - Statement on retention duration relative to the disclosed purpose.

### 7. Security Measures
- **GDPR**:
  - Evaluation of risks inherent in data processing.
  - Implementation of measures to mitigate risks.
  - Appropriate level of security, including confidentiality.
  - Technical and organizational measures (e.g., encryption).
- **CCPA**:
  - Reasonable security procedures and practices.
  - Protection against unauthorized or illegal access, destruction, use, modification, or disclosure.

### 8. Automated Decision-Making and Profiling
- **GDPR**:
  - Transparency and information about automated decision-making.
  - Legal basis for automated decision-making.
  - Rights of data subjects (human intervention, express point of view, contest decision).
  - Safeguards for fairness, transparency, and non-discrimination.
  - Data minimization and accuracy.
  - Data Protection Impact Assessments (DPIAs).
  - Security measures.
  - Regular reviews.
- **CCPA**:
  - Opt-out links for sale or sharing of personal information.
  - Description of consumer rights.
  - Mechanism for exercising rights without creating an account.
  - Use of personal information solely for compliance with requests.

### 9. Cookies and Tracking Technologies
- **GDPR**:
  - Types of cookies and tracking technologies used.
  - Purpose of each cookie and tracking technology.
  - Data collected.
  - Legal basis for processing.
  - Duration of data storage.
  - Third-party access.
  - User rights.
  - Instructions for managing cookies.
  - Impact of disabling cookies.
  - Contact information.
- **CCPA**:
  - Description of consumer rights regarding cookies and tracking technologies.
  - Opt-out links for sale or sharing of personal information.
  - Information on financial incentives related to personal information.

### 10. Changes to the Privacy Policy
- **GDPR**:
  - Notification of changes.
  - Effective date of changes.
  - Description of changes.
  - User rights regarding changes.
  - Access to previous versions.
  - Contact information.
- **CCPA**:
  - Disclosure of consumer rights.
  - Procedures for requests.
  - Business obligations.
  - Frequency and circumstances for requests.
  - Opt-out information.
  - Updates and notifications.
  - Accessibility.

### 11. Contact Information
- **GDPR**:
  - Identity of the controller.
  - Existence of the processing operation.
  - Purposes of the processing.
  - Right to lodge a complaint.
  - Right to request access to and rectification or erasure of personal data or restriction of processing.
  - Legal basis for processing.
  - Data storage duration.
- **CCPA**:
  - Contact information for handling consumer inquiries.
  - Opt-out links for sale or sharing of personal information.
  - Links to limit the use or disclosure of sensitive personal information.</b>

In [13]:
# from llama_index.postprocessor.cohere_rerank import CohereRerank
from llama_index.core.response_synthesizers import TreeSummarize
from llama_index.core.query_pipeline import QueryPipeline
from llama_index.core import PromptTemplate

# define modules
prompt_str = "Please generate a question about Paul Graham's life regarding the following topic {topic}"
prompt_tmpl = PromptTemplate(prompt_str)
llm = OpenAI(model="gpt-3.5-turbo")
retriever = SummaryIndex.from_documents([regulation_text[0]
                                         ]).as_retriever(similarity_top_k=3)
# reranker = CohereRerank()
summarizer = TreeSummarize(llm=llm)

In [ ]:
index = SummaryIndex.from_documents([regulation_text[0]])

In [9]:
synthesizer = get_response_synthesizer(llm=llm,
                                       response_mode=ResponseMode.COMPACT)

In [ ]:
index.as_retriever().retrieve(
    "What sections should a GDPR compliant privacy policy include? And for each section, what key points should be covered?"
)

[NodeWithScore(node=TextNode(id_='8c8e6d71-525e-431a-9e1f-5265fba77978', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='https://eur-lex.europa.eu/legal-content/EN/TXT/HTML/?uri=CELEX:32016L0680', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='caf142112d0f6a347f255b21c17bed3b471f6c85a139a8ffc943ead6774c8819'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='738348da-0799-4937-9547-8bee98317c52', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='ca82bc8311cc6b2e31c37be9b4f045d8b8594ac366ae78e8f72c551c3520a820')}, text='4.5.2016\n\n|\n\nEN\n\n|\n\nOfficial Journal of the European Union\n\n|\n\nL 119/89  \n  \n---|---|---|---  \n  \n* * *\n\nDIRECTIVE (EU) 2016/680 OF THE EUROPEAN PARLIAMENT AND OF THE COUNCIL\n\nof 27 April 2016\n\non the protection of natural persons with regard to the processing of personal\ndata by competent authorities for the purpo

In [19]:
synthesizer.get_response?

Signature:
synthesizer.get_response(
    query_str: str,
    text_chunks: Sequence[str],
    prev_response: Union[pydantic.v1.main.BaseModel, str, Generator[str, NoneType, NoneType], AsyncGenerator[str, NoneType], NoneType] = None,
    **response_kwargs: Any,
) -> Union[pydantic.v1.main.BaseModel, str, Generator[str, NoneType, NoneType], AsyncGenerator[str, NoneType]]
Docstring: Get compact response.
File:      d:\program\miniconda3\envs\dl\lib\site-packages\llama_index\core\response_synthesizers\compact_and_refine.py
Type:      BoundFunctionWrapper

## Judge Generator (RAW)

In [ ]:
JUDGE_SECTION_TEMPLATE = """\
This {section_name} is a section of a privacy policy:

{section_text}

Please judge whether this section is compliant with the {regulations} regulations. If not, please provide suggestions on how to improve it.
NOTE THAT this is just one section, not the whole privacy policy. Only judge this section in isolation.
"""
FORMAT_JUDGE_TEMPLATE = """\
Given the comments from a legal expert:

{query_str}

Please extract the legal expert's suggestions on how to improve the sections in a privacy policy.
"""

In [ ]:
class Judge(BaseModel):

    name: SectionNames = Field(..., title="Section Name")

    suggestions: str = Field(
        ...,
        title="Suggestions",
        description="Suggestions for improvement.",
    )


class Judges(BaseModel):

    judges: List[Judge] = Field(
        ...,
        title="Judges",
        description="Judges for the sections.",
    )
    # suggestions: Dict[SectionNames, str] = Field(
    #     ...,
    #     title="Suggestions",
    #     description="Suggestions for sections improvement.",
    # )


# class Judges2(BaseModel):
#     improvements: Dict[SectionNames, str] = Field(
#         ...,
#         title="improvements",
#         description="Suggestions for each section's improvement.",
#     )

In [ ]:
judge_section_template = PromptTemplate(JUDGE_SECTION_TEMPLATE)
judge_parser = PydanticOutputParser(Judges)
format_judge_template = PromptTemplate(
    judge_parser.format(FORMAT_JUDGE_TEMPLATE))

In [54]:
test_func = FnComponent(
    fn=lambda input, section_name: f"{len(input.judges)}, {len(section_name)}")
# {
#     "section_name": section_name,
#     "section_text": section_text,
#     "regulations": regulations,
# },

In [63]:
def determine_judge(judges: Judges, prompt: str):

    section_name = ''
    for section in SectionNames:
        if section.value in prompt:
            section_name = section.value

    if not section_name:
        raise ValueError("Section name not found in prompt.")

    for judge in judges.judges:
        if judge.name.value == section_name:
            return {"pass": False, "suggestions": judge.suggestions}

        return {"pass": True, "suggestions": ""}


In [66]:
q = QueryPipeline(verbose=True)
q.add_modules({
    "judge_section_template": judge_section_template,
    "law_llm": law_llm,
    "format_judge_template": format_judge_template,
    "llm": Settings.llm,
    "judge_parser": judge_parser,
    "determine_judge": FnComponent(fn=determine_judge),
})
q.add_link("judge_section_template", "law_llm")
q.add_link("law_llm", "format_judge_template")
q.add_link("format_judge_template", "llm")
q.add_link("llm", "judge_parser")
q.add_link("judge_parser", "determine_judge", dest_key="judges")
q.add_link("judge_section_template", "determine_judge", dest_key="prompt")

In [56]:
q.module_dict

{'judge_section_template': PromptComponent(partial_dict={}, prompt=PromptTemplate(metadata={'prompt_type': <PromptType.CUSTOM: 'custom'>}, template_vars=['section_name', 'section_text', 'regulations'], kwargs={}, output_parser=None, template_var_mappings=None, function_mappings=None, template='This {section_name} is a section of a privacy policy:\n\n{section_text}\n\nPlease judge whether this section is compliant with the {regulations} regulations. If not, please provide suggestions on how to improve it.\nNOTE THAT this is just one section, not the whole privacy policy. Only judge this section in isolation.\n'), llm=None, format_messages=False),
 'law_llm': LLMCompleteComponent(partial_dict={}, llm=HuggingFaceLLMModified(callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x0000011D5391D420>, system_prompt='', messages_to_prompt=<function prepare_law_llm.<locals>.messages_to_prompt at 0x0000011BCAE2BAC0>, completion_to_prompt=<function default_completion_to_prom

In [67]:
output = q.run(
    section_name="Data Collection and Usage",
    section_text=generated_section,
    regulations="GDPR, CCPA",
)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


> Running module judge_section_template with input: 
section_name: Data Collection and Usage
section_text: The data we collect depends on the context of your interactions with Microsoft and the choices you make (including your privacy settings), the products and features you use, your location, and applica...
regulations: GDPR, CCPA

> Running module law_llm with input: 
prompt: This Data Collection and Usage is a section of a privacy policy:

The data we collect depends on the context of your interactions with Microsoft and the choices you make (including your privacy settin...

> Running module format_judge_template with input: 
query_str: This section appears to be compliant with the GDPR and CCPA regulations. It provides clear information about the data collected by Microsoft, the purposes for which the data is used, and the legal bas...

> Running module llm with input: 
messages: Given the comments from a legal expert:

This section appears to be compliant with the GDPR and CCPA 

In [68]:
output

{'pass': False,
 'suggestions': 'Clarify the legal basis for processing each type of data collected, such as consent, contract, or legitimate interest.'}

In [50]:
output

'4, 1911'

In [52]:
judge_pipeline = QueryPipeline(chain=[
    judge_section_template,
    law_llm,
    format_judge_template,
    Settings.llm,
    judge_parser,
    test_func,
],
                               verbose=True)

In [44]:
module_names = list(judge_pipeline.module_dict.keys())

In [45]:
judge_pipeline.add_link(module_names[0], module_names[-1], dest_key="prompt")

In [46]:
output = judge_pipeline.run(
    section_name="Data Collection and Usage",
    section_text=generated_section,
    regulations="GDPR, CCPA",
)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


> Running module c950a9cc-1007-47c8-81be-975e7d5a584c with input: 
section_name: Data Collection and Usage
section_text: The data we collect depends on the context of your interactions with Microsoft and the choices you make (including your privacy settings), the products and features you use, your location, and applica...
regulations: GDPR, CCPA

> Running module 17259d05-5c9b-49e8-a997-b6ef220dd202 with input: 
prompt: This Data Collection and Usage is a section of a privacy policy:

The data we collect depends on the context of your interactions with Microsoft and the choices you make (including your privacy settin...

> Running module 26206de1-b0cc-4aac-b394-06e0d33ed296 with input: 
query_str: This section appears to be compliant with the GDPR and CCPA regulations. It provides clear information about the data collected by Microsoft, the purposes for which the data is used, and the legal bas...

> Running module 202dc140-19cc-4b05-9146-b71c3102094c with input: 
messages: Given the 

ValueError: Module input keys must have exactly one key if dest_key is not specified. Remaining keys: in module: {'input', 'prompt'}

In [ ]:
output.judges

[Judge(name=<SectionNames.SECTION3: 'Data Collection and Usage'>, suggestions='Clarify the legal basis for processing each type of data collected, such as consent, contract, or legitimate interest.'),
 Judge(name=<SectionNames.SECTION6: 'Data Retention'>, suggestions='Include information about how long Microsoft retains the data and the criteria used to determine the retention period.'),
 Judge(name=<SectionNames.SECTION4: 'Data Subject Rights'>, suggestions='Provide information about how data subjects can exercise their rights to access, rectify, erase, restrict, and object to processing.'),
 Judge(name=<SectionNames.SECTION5: 'Data Sharing and Transfers'>, suggestions='Ensure that data is shared only on a need-to-know basis and provide details about the entities with whom data is shared.')]